In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np
from sklearn import linear_model
import pandas as pd 
from ydata_profiling import ProfileReport

In [24]:
file_path_train = "C:/Users/jeyas/OneDrive/Documents/CSE158/HeartAttackPrediction/Datasets/diabetes_train.csv"
file_path_test = "C:/Users/jeyas/OneDrive/Documents/CSE158/HeartAttackPrediction/Datasets/diabetes_test.csv"
dataTrain = pd.read_csv(file_path_train) #, index_col = "Unnamed:0")
dataTest = pd.read_csv(file_path_test)
dataTest.rename(columns = {"Unnamed: 0": "index"}, inplace = True)
dataTrain.rename(columns = {"Unnamed: 0": "index"}, inplace = True)

Logistic Regression Model

In [22]:
def accuracy(predictions, y):
    correct = predictions == y # Binary vector indicating which predictions were correct
    return sum(correct) / len(correct)

In [23]:
def BER(pred, y):
    TP = np.sum([(p and l) for (p,l) in zip(pred, y)])
    FP = np.sum([(p and not l) for (p,l) in zip(pred, y)])
    TN = np.sum([(not p and not l) for (p,l) in zip(pred, y)])
    FN = np.sum([(not p and l) for (p,l) in zip(pred, y)])
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    BER = 1 - 1/2 * (TPR + TNR)
    return BER

In [38]:
dataTrain.columns

Index(['index', 'gender', 'age', 'hypertension', 'heart_disease',
       'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level',
       'diabetes'],
      dtype='object')

In [39]:
def feature(row, dataTrain = dataTrain):
    feat1 = dataTrain.loc[row]['HbA1c_level']
    feat2 = dataTrain.loc[row]['blood_glucose_level']
    return [1] + [feat1] + [feat2]

In [40]:
X_train = [feature(row) for row in range(len(dataTrain))]
y_train = [dataTrain.loc[row]['diabetes'] for row in range(len(dataTrain))]
X_test = [feature(row) for row in range(len(dataTest))]
Y_test = [dataTest.loc[row]['diabetes'] for row in range(len(dataTest))]

In [41]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(X_train,y_train)
pred = mod.predict(X_test)

In [42]:
BER(pred, Y_test)

0.5054605649317756

Baseline Model Based on Threshold with HbA1c Levels

In [18]:
best_accuracy = 0 
best_threshold = 0
for threshold in np.arange(6.5, 6.7, 0.01):
    ypred = [1 if dataTrain.loc[row]['HbA1c_level'] > threshold else 0 for row in dataTrain.index]
    correct = [1 if dataTrain.loc[row]['diabetes'] == pred else 0 for pred,row in zip(ypred, dataTrain.index)]
    accuracy = sum(correct)/len(correct)
    if accuracy > best_accuracy:
        best_accuracy = accuracy 
        best_threshold = threshold 
    print(best_threshold)


6.5
6.5
6.5
6.5
6.5
6.5
6.5
6.5
6.5
6.5
6.5
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998
6.609999999999998


In [25]:
ypred = [1 if dataTest.loc[row]['HbA1c_level'] > best_threshold else 0 for row in dataTest.index]
correct = [1 if dataTest.loc[row]['diabetes'] == pred else 0 for pred,row in zip(ypred, dataTest.index)]
y = [dataTest.loc[row]['diabetes'] for row in dataTest.index]
accuracy = sum(correct)/len(correct)
print(accuracy)

0.9529


In [27]:
BER(ypred, y)

0.2700688073394495